In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [7]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True
)

13568290/13568290 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
csvpath,_ = os.path.splitext(zip_path)

In [9]:
df = pd.read_csv(csvpath)

In [10]:
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420551 entries, 0 to 420550
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date Time        420551 non-null  object 
 1   p (mbar)         420551 non-null  float64
 2   T (degC)         420551 non-null  float64
 3   Tpot (K)         420551 non-null  float64
 4   Tdew (degC)      420551 non-null  float64
 5   rh (%)           420551 non-null  float64
 6   VPmax (mbar)     420551 non-null  float64
 7   VPact (mbar)     420551 non-null  float64
 8   VPdef (mbar)     420551 non-null  float64
 9   sh (g/kg)        420551 non-null  float64
 10  H2OC (mmol/mol)  420551 non-null  float64
 11  rho (g/m**3)     420551 non-null  float64
 12  wv (m/s)         420551 non-null  float64
 13  max. wv (m/s)    420551 non-null  float64
 14  wd (deg)         420551 non-null  float64
dtypes: float64(14), object(1)
memory usage: 48.1+ MB


In [12]:
df.describe()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
count,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000,420551.000000
mean,989.212776,9.450147,283.492743,4.955854,76.008259,13.576251,9.533756,4.042412,6.022408,9.640223,1216.062748,1.702224,3.056555,174.743738
std,8.358481,8.423365,8.504471,6.730674,16.476175,7.739020,4.184164,4.896851,2.656139,4.235395,39.975208,65.446714,69.016932,86.681693
min,913.600000,-23.010000,250.600000,-25.010000,12.950000,0.950000,0.790000,0.000000,0.500000,0.800000,1059.450000,-9999.000000,-9999.000000,0.000000
25%,984.200000,3.360000,277.430000,0.240000,65.210000,7.780000,6.210000,0.870000,3.920000,6.290000,1187.490000,0.990000,1.760000,124.900000
50%,989.580000,9.420000,283.470000,5.220000,79.300000,11.820000,8.860000,2.190000,5.590000,8.960000,1213.790000,1.760000,2.960000,198.100000
75%,994.720000,15.470000,289.530000,10.070000,89.400000,17.600000,12.350000,5.300000,7.800000,12.490000,1242.770000,2.860000,4.740000,234.100000
max,1015.350000,37.280000,311.340000,23.110000,100.000000,63.770000,28.320000,46.010000,18.130000,28.820000,1393.540000,28.490000,23.500000,360.000000


In [13]:
df.shape

(420551, 15)

In [17]:
df.isna().sum()

,0
Date Time,0
p (mbar),0
T (degC),0
Tpot (K),0
Tdew (degC),0
rh (%),0
VPmax (mbar),0
VPact (mbar),0
VPdef (mbar),0
sh (g/kg),0


In [19]:
df["Date Time"].value_counts()

,count
Date Time,
21.03.2014 12:50:00,2
01.07.2010 03:50:00,2
01.07.2010 04:10:00,2
01.07.2010 04:20:00,2
01.07.2010 04:30:00,2
...,...
01.09.2011 09:50:00,1
01.09.2011 09:40:00,1
01.09.2011 09:30:00,1


In [46]:
# consider only 2009 data, i.e. first 52557 rows

In [35]:
df=df.iloc[:52557]

In [47]:
# scale column to be in same range

In [40]:
df1 = df[['T (degC)']].dropna()

In [42]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df1), columns=df1.columns)

In [45]:
# dot shuffle, since time series order is important

In [44]:
train,test = train_test_split(df_scaled, test_size=0.2,shuffle=False)

In [48]:
model = Sequential()

In [49]:
model.add(LSTM(50, activation='relu', input_shape=(None,1)))
model.add(Dense(1))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [50]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [51]:
windowsize = 10
x_train = np.array([train.iloc[i-windowsize:i].values for i in range(windowsize, len(train))])

In [53]:
y_train = train.iloc[windowsize:, 0].values

In [54]:
y_train

array([0.25450973, 0.25236649, 0.2529023 , ..., 0.39364172, 0.39006966,
       0.38846223])

In [55]:
x_test = np.array([test.iloc[i-windowsize:i].values for i in range(windowsize, len(test))])

In [56]:
y_test = test.iloc[windowsize:, 0].values

In [57]:
x_train = x_train.reshape(x_train.shape[0], windowsize,1)

In [58]:
x_test = x_test.reshape(x_test.shape[0], windowsize,1)

In [59]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0319 - val_loss: 4.2709e-05
Epoch 2/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 8.2806e-05 - val_loss: 3.9389e-05
Epoch 3/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 7.1838e-05 - val_loss: 3.1596e-05
Epoch 4/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 5.9734e-05 - val_loss: 2.4150e-05
Epoch 5/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 5.6059e-05 - val_loss: 2.5182e-05
Epoch 6/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 5.1129e-05 - val_loss: 2.1419e-05
Epoch 7/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 4.5554e-05 - val_loss: 2.1126e-05
Epoch 8/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 3.8894e-05 - val_loss: 1.5053e-05
Epoch 9/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 3.8660e-05 - val_loss: 1.3637e-05
Epoch 10/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 3.1822e-05 - val_loss: 1.2553e-05


In [60]:
trainloss = model.evaluate(x_train, y_train)
testloss = model.evaluate(x_test, y_test)

1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.3974e-05
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3878e-05


In [61]:
recent_data = np.array(df_scaled['T (degC)'].tail(windowsize)).reshape(1, windowsize,1)

In [62]:
predict = model.predict(recent_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step


In [63]:
predicted_value = scaler.inverse_transform(predict)

In [64]:
actual = np.array(df['T (degC)'].iloc[-1]).reshape(-1,1)

In [66]:
print(predicted_value)
print(actual)

[[-2.8045347]]
[[-2.82]]


In [69]:
recent_data

array([[[0.36881586],
        [0.36774424],
        [0.36756564],
        [0.36631541],
        [0.36595821],
        [0.36470798],
        [0.36345776],
        [0.36274335],
        [0.36202893],
        [0.36060011]]])